Here is my first stab at using the Census API wrapper. I'm grabbing the 1-year American Community Survey results here, but this can be adjusted to several different looks if you peak at the 'Datasets' section on this website: https://github.com/datamade/census/blob/master/README.rst
I haven't figured out how to name dataframes based on the iterated 'year' in years, so the only dataframe that ends up being captured in my code is the last year in the set of 'years' which is 2018.
Other than that, I've formatted the dataframe. There are lots of data fields, but I wanted to cast a large net, so we can check out the fields and reduce from there. Let me know if you all have notes on this for improvement!

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
# Added 'census' and 'us' to the dependency list to utilize the census wrapper
from census import Census
from us import states

# Import API key
from config import api_key

c = Census(api_key)

In [2]:
# Naming the data points for easier reading within the loop/API request
median_income = 'B19013_001E'
employed = 'B23025_004E'
unemployed = 'B23025_005E'
poverty = 'B17001_002E'
poverty_family = 'B17012_002E'
age = 'B01002_001E'
population = 'B01003_001E'
pop_white_alone = 'B02001_002E'
pop_black_alone = 'B02001_003E'
pop_american_indian_alone = 'B02001_005E'
pop_native_hawaiian_alone = 'B02001_006E'
pop_two_or_more_races = 'B02001_007E'
pop_hispanic_origin = 'B02001_008E'
median_home_value = 'B25077_001E'
median_gross_rent = 'B25064_001E'
commute_time_pub_transit = 'B08136_007E'
commute_time_solo_auto = 'B08136_003E'
commute_time_walked = 'B08136_011E'
transit_solo_auto = 'B08301_003E'
transit_pub_transit = 'B08301_010E'
transit_walked = 'B08301_019E'
transit_other = 'B08101_041E'
ed_none = 'B15003_002E'

In [3]:
# Starting the for loop to pull in the data fields. Having some trouble naming the four dataframes using the 'year' iterable...
# Any thoughts on how to do that?
# Run the for loop and it will give you the last dataframe in the series which is '2018', currently. I've formatted the df.
# This is almost certainly more information than we need, but I wanted to include data fields that might be relevent.
# We can always shrink down the dataframes to the fields we want to focus on.
years = [2011,2012,
       2014,
       2016,
       2018]
for year in years:
    med_income = c.acs1.state(('NAME',
                              median_income,
                              employed,
                              unemployed,
                              poverty,
                              poverty_family,
                              age,
                              population,
                              pop_white_alone,
                              pop_black_alone,
                              pop_american_indian_alone,
                              pop_native_hawaiian_alone,
                              pop_two_or_more_races,
                              pop_hispanic_origin,
                              median_home_value,
                              median_gross_rent,
                              commute_time_solo_auto,
                              commute_time_pub_transit,
                              commute_time_walked,
                              transit_solo_auto,
                              transit_pub_transit,
                               # 'year=year' is where the 'year in years' is iterated. You can replace this with whatever
                               # year you want, take it out of for loop and run.
                              transit_walked),Census.ALL,year=year)
    census_df = pd.DataFrame(med_income)
    census_df = census_df.rename(columns={
        'B19013_001E':'Median Income',
        'B23025_004E':'Number Employed',
        'B23025_005E':'Number Unemployed',
        'B17001_002E':'Number Poverty',
        'B17012_002E':'Number Families in Poverty',
        'B01002_001E':'Median Age',
        'B01003_001E':'Population',
        'B02001_002E':'Pop: White Only',
        'B02001_003E':'Pop: Black Only',
        'B02001_005E':'Pop: American Indian Only',
        'B02001_006E':'Pop: Native Hawaiian Only',
        'B02001_007E':'Pop: Two or More Races',
        'B02001_008E':'Pop: Hispanic Origin',
        'B25077_001E':'Median Home Value',
        'B25064_001E':'Median Gross Rent',
        'B08136_007E':'Commute Time: Solo Auto',
        'B08136_003E':'Commute Time: Public Transit',
        'B08136_011E':'Commute Time: Walking',
        'B08301_003E':'Transit: Solo Auto',
        'B08301_010E':'Transit: Public Transit',
        'B08301_019E':'Transit: Walking'
    })
    census_df['Median Income'] = census_df['Median Income'].astype(float).map("${:,.2f}".format)
    census_df['Median Home Value'] = census_df['Median Home Value'].astype(float).map("${:,.2f}".format)
    census_df['Median Gross Rent'] = census_df['Median Gross Rent'].astype(float).map("${:,.2f}".format)
    census_df['Umemployment Rate'] = (census_df['Number Unemployed']/(census_df['Number Employed']+census_df['Number Unemployed'])).astype(float).map("{:.2%}".format)
    census_df['Poverty Rate'] = (census_df['Number Poverty']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Commute Time: Public Transit'] = (census_df['Commute Time: Public Transit']/census_df['Population'])
    census_df['Commute Time: Solo Auto'] = (census_df['Commute Time: Solo Auto']/census_df['Population'])
    census_df['Commute Time: Walking'] = (census_df['Commute Time: Walking']/census_df['Population'])
    census_df['Transit: Solo Auto'] = (census_df['Transit: Solo Auto']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Transit: Public Transit'] = (census_df['Transit: Public Transit']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Transit: Walking'] = (census_df['Transit: Walking']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Number Employed'] = census_df['Number Employed'].astype(int)
    census_df['Number Unemployed'] = census_df['Number Unemployed'].astype(int)
    census_df['Number Poverty'] = census_df['Number Poverty'].astype(int)
    census_df['Number Families in Poverty'] = census_df['Number Families in Poverty'].astype(int)
    census_df['Population'] = census_df['Population'].astype(int)
    census_df['Pop: White Only'] = census_df['Pop: White Only'].astype(int)
    census_df['Pop: Black Only'] = census_df['Pop: Black Only'].astype(int)
    census_df['Pop: American Indian Only'] = census_df['Pop: American Indian Only'].astype(int)
    census_df['Pop: Native Hawaiian Only'] = census_df['Pop: Native Hawaiian Only'].astype(int)
    census_df['Pop: Two or More Races'] = census_df['Pop: Two or More Races'].astype(int)
    census_df['Pop: Hispanic Origin'] = census_df['Pop: Hispanic Origin'].astype(int)
    census_df['Pop Rate: White Only'] = (census_df['Pop: White Only']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Pop Rate: Black Only'] = (census_df['Pop: Black Only']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Pop Rate: American Indian Only'] = (census_df['Pop: American Indian Only']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Pop Rate: Native Hawaiian Only'] = (census_df['Pop: Native Hawaiian Only']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Pop Rate: Two or More Races'] = (census_df['Pop: Two or More Races']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Pop Rate: Hispanic Origin'] = (census_df['Pop: Hispanic Origin']/census_df['Population']).astype(float).map("{:.2%}".format)
census_df

UnsupportedYearException: Geography is not available in 1990. Available years include (2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011)